## WARNING!
This Process takes a very long time and is very memory heavy.

In [ ]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

from IPython.display import clear_output

In [ ]:
data = pd.read_csv('../Data/Processed Data/complete_data.csv')

In [ ]:
def normalize(col_data):
    return (col_data - col_data.min())/(col_data.max()-col_data.min())

In [ ]:
def update_progress(actual, total, clear=True, title="Progress"):
    bar_length = 100

    progress = (actual / total)

    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    if clear:
        clear_output(wait = True)

    text = "{0}: [{1}] {2:.1f}%".format(title, "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [ ]:
if data is None or len(data) <= 0:
    raise Error("Data has not been provided or an empty array has been given.")
    
MAX_ACIDS = 2000

acid_columns = np.arange(0, MAX_ACIDS, 1)
X = pd.DataFrame([], columns=acid_columns)
y = pd.DataFrame()

sequences = pd.DataFrame(data['Sequence'], columns=['Sequence'])
y = pd.DataFrame(data['Solubility(%)'])

total_seq = len(sequences)
i_seq = 0

for idx, row in sequences.iterrows():
    sequence = row.iloc[0]
    
    total_acid = len(sequence)
    i_acid = 0
    
    amino_acid_arr = np.zeros(MAX_ACIDS)
    
    for acid in sequence:
        acid_num_val = ord(acid)
        amino_acid_arr[i_acid] = acid_num_val
        i_acid += 1
        update_progress(i_seq, total_seq, clear=True, title='Sequences Progress')
        update_progress(i_acid, total_acid, title="Current Sequence Amino Acid Progress", clear=False)
    
    X_dict = dict(zip(acid_columns, amino_acid_arr))
    
    X = pd.concat([X, pd.DataFrame([X_dict], columns=acid_columns)], ignore_index=True)
    i_seq += 1
    update_progress(i_seq, total_seq, clear=True, title='Sequences Progress')

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# Add Solubility Data to the data
export_data = X.copy()
export_data['UP_ID'] = pd.DataFrame(data['UP_ID'])

# # Move Solubility as the first column
export_data = export_data[ ['UP_ID'] + [ col for col in export_data.columns if col != 'UP_ID' ] ]

In [ ]:
merged_export_data = data.merge(export_data, how='inner', on='UP_ID')

In [ ]:
merged_export_data.to_csv('sequenced_data.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8ea8948f-b978-492f-a6dd-8ac5feb4b472' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>